# possible places to improve

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
### =====================================================
#train and possibly use parallel(maybe)
### =====================================================


from sklearn.linear_model import LinearRegression, Ridge
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
### =====================================================
#adjust model#
### =====================================================


from sklearn.metrics import mean_squared_error, r2_score
### =====================================================
#adjust loss function
### =====================================================


# ===== Deep Learning =====
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# ===== Reproducibility =====
np.random.seed(42)
torch.manual_seed(42)


In [ ]:
### =====================================================
# TODO: Replace with real data
### =====================================================

def load_data():
    X = np.empty((0, 10))   #  10 Purkinje / pupil features
    y = np.empty((0,))     #  accommodation depth
    return X, y


X, y = load_data()

# check
assert X.ndim == 2
assert y.ndim in [1, 2]


In [ ]:
def evaluate_regression(y_true, y_pred, name="model"):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    print(f"[{name}] RMSE = {rmse:.4f}, R2 = {r2:.4f}")
    return rmse, r2

# NORMAL






In [ ]:
def run_baseline_model(model_type,X_train,y_train,X_test,y_test):


    if model_type == "linear":
        model, name = Pipeline([("scaler", StandardScaler()), ("reg", Ridge(alpha=1.0))]), "Linear / Ridge"
    elif model_type == "svm":
        model, name = Pipeline([("scaler", StandardScaler()),("svr", SVR(kernel="rbf", C=10.0, epsilon=0.01, gamma="scale"))]), "SVR (RBF)"
    elif model_type == "tree":
        model, name = DecisionTreeRegressor(max_depth=5, random_state=42), "Decision Tree"
    elif model_type == "forest":
        model, name = RandomForestRegressor(n_estimators=200, max_depth=8, random_state=42), "Random Forest"
    elif model_type == "mlp":
        model, name = Pipeline([("scaler", StandardScaler()),
                             ("mlp", MLPRegressor(hidden_layer_sizes=(64,64), max_iter=1000))]), "MLP"

    else:
        raise ValueError(f"Unknown model_type: {model_type}")


    # ===== Train & Evaluate =====
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    evaluate_regression(y_test, y_pred, name=name)
    return model


# MLP

In [ ]:
class TabularDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32).unsqueeze(-1)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
def train_mlp(X_train, y_train, X_test, y_test, epochs=100):
    train_ds = TabularDataset(X_train, y_train)
    test_ds = TabularDataset(X_test, y_test)

    train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)

    model = MLP(input_dim=X_train.shape[1])
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    loss_fn = nn.MSELoss()

    for epoch in range(epochs):
        model.train()
        for xb, yb in train_loader:
            pred = model(xb)
            loss = loss_fn(pred, yb)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    model.eval()
    with torch.no_grad():
        y_pred = model(torch.tensor(X_test, dtype=torch.float32)).numpy().squeeze()

    evaluate_regression(y_test, y_pred, name="MLP")
    return model


# CNN

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self, out_dim=1):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((4, 4))
        )
        self.fc = nn.Linear(16 * 4 * 4, out_dim)

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)


In [ ]:
np.random.seed(0)

X = np.random.randn(300, 6)
y = (
    2.0 * X[:, 0]
    - 1.5 * X[:, 1]
    + 0.5 * X[:, 2] ** 2
    + 0.1 * np.random.randn(300)
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)
for m in ["linear", "svm", "tree", "forest", "mlp"]:
    run_baseline_model(m, X_train, y_train, X_test, y_test)

print("X_train shape:", X_train.shape, "X_test shape:", X_test.shape)
print("y_train range:", (y_train.min(), y_train.max()))
print("y_test  range:", (y_test.min(), y_test.max()))
print("X_train mean/std:", (X_train.mean(), X_train.std()))
print("X_test  mean/std:", (X_test.mean(), X_test.std()))

print("y_train min/max:", y_train.min(), y_train.max())
print("y_test  min/max:", y_test.min(), y_test.max())



[Linear / Ridge] RMSE = 0.7490, R2 = 0.9182
[SVR (RBF)] RMSE = 0.7179, R2 = 0.9248
[Decision Tree] RMSE = 1.2768, R2 = 0.7622
[Random Forest] RMSE = 0.9754, R2 = 0.8612
[MLP] RMSE = 0.2818, R2 = 0.9884
X_train shape: (210, 6) X_test shape: (90, 6)
y_train range: (np.float64(-5.746499945829033), np.float64(9.343694689936145))
y_test  range: (np.float64(-6.221487734079538), np.float64(6.867591313824606))
X_train mean/std: (np.float64(-0.028323515686789997), np.float64(0.9643740646507118))
X_test  mean/std: (np.float64(0.019857510831474245), np.float64(1.0249922345652773))
y_train min/max: -5.746499945829033 9.343694689936145
y_test  min/max: -6.221487734079538 6.867591313824606


Input Features (X)

The input variable X represents a set of low-dimensional, physically meaningful features extracted from the smart glass sensing system at each time frame. These features encode the geometric and optical characteristics of the eye, including the two-dimensional positions of Purkinje reflections (e.g., P1, P3, and P4), the pupil center location, and pupil size-related measurements. Rather than operating on raw image data, the feature representation embeds prior knowledge of ocular optics and eye physiology, effectively constraining the learning problem to a compact and interpretable input space. As a result, the model focuses on learning residual nonlinear relationships caused by individual anatomical differences, sensor placement variability, and non-ideal optical effects.

Target Variable (y)

The target variable y corresponds to the accommodation state associated with each observation, which may be represented by accommodation distance (e.g., in diopters), experimentally controlled focal depth, or a calibrated proxy derived from dynamic accommodation measurements. This target reflects the latent visual state that the smart glass system aims to infer in real time. By learning a mapping from observable optical features to accommodation state, the system enables continuous and non-invasive estimation of visual focus dynamics during natural viewing conditions.

1️⃣ Calibration（个体校准）

你可以做：

每个受试者单独 fine-tune（few-shot）

在 X 中加入 subject-specific bias

用 linear / MLP 做 post-calibration mapping


2️⃣ Data training（鲁棒训练）

你可以做：

加噪声模拟传感器抖动

数据归一化（scale by pupil size）

LOSO（leave-one-subject-out）评估

